### NLP Assignment 2, Language Model
## Ashish Kumar Sinha
## MDS201904

In [7]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
import re
import fileinput
import collections
import nltk
from nltk import ngrams
import pickle
import json
import csv
import operator
import random
import math
from numba import jit, cuda
from IPython.display import Image
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
#import spacy
#from spacy_langdetect import LanguageDetector
#nlp = spacy.load('en')
#nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

! pip install langid
import langid

import gc

! pip install in_place
import in_place

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
file_path = 'data/initial_corpus.txt'

## Removing spaces before "."

In [25]:
def remove_spaces(file_path):
    """
    args: file_path - file path of the corpus
    returns: converts text to lower case
    """ 
    print('---------------------------------------------------------')
    print("removing exptra spaces")
    with in_place.InPlace(file_path, encoding = 'utf-8') as file:
        for line in tqdm(file):  
            line = re.sub(r'\s+\.', '. ', line)
            line = re.sub(r'(\.)(\w+)',r'\1 \2', line)
            file.write(line)
    gc.collect()
    print('Done')
    print('---------------------------------------------------------')

In [26]:
remove_spaces(file_path)

51it [00:00, 491.07it/s]

---------------------------------------------------------
removing exptra spaces


52817it [01:59, 441.30it/s]


Done
---------------------------------------------------------


## Sentence Tokenizing and writing one sentence per line

In [8]:
def sentence_tokenize(file_path):
    """
    args: file_path - file path of text corpus
    returns: writes one sentence per line
    """
    print('---------------------------------------------------------')
    print("sentence tokenizing")
    pat = re.compile(r'([A-Z][^\.!?]*[\.!?])', re.M)
    with open('data/preprocessed.txt', 'w', encoding='utf-8') as f2:
        with open(file_path, 'r', encoding='utf-8') as f1:
            for i,line in tqdm(enumerate(f1)):
                # print(line)
                sentences = sent_tokenize(line)
                # print(sentences)
                for sentence in sentences:
                    f2.write(sentence)
                    f2.write('\n')
                    
    print('Done')
    print('---------------------------------------------------------')

In [9]:
sentence_tokenize(file_path)

0it [00:00, ?it/s]

---------------------------------------------------------
sentence tokenizing


52817it [04:44, 185.49it/s]

Done
---------------------------------------------------------


## Converting preprocessed corpus to lower case, removing ending punctuation ('.', '?', etc) and delimiting sentences

In [5]:
def to_lower(file_path):
    """
    args: file_path - file path of the corpus
    returns: converts text to lower case
    """
    print('---------------------------------------------------------')
    print("converting to lower case and adding delimiters")
    with in_place.InPlace(file_path, encoding = 'utf-8') as file:
        for line in tqdm(file):  
            if len(line) > 1:
                line = line.lower()
                file.write(line[:-2])
                file.write('\n')
    gc.collect()
    print('Done')
    print('---------------------------------------------------------')
    with in_place.InPlace(file_path, encoding = 'utf-8') as file:
        for line in tqdm(file):  
            if len(line) > 50:
                file.write(line)
                #file.write('\n')
    gc.collect()

In [6]:
file_path = 'data/preprocessed.txt'

In [7]:
to_lower(file_path)

41653it [00:00, 413469.52it/s]

---------------------------------------------------------
converting to lower case and adding delimiters


9039055it [00:20, 450860.40it/s]
57465it [00:00, 570483.08it/s]

Done
---------------------------------------------------------


9039055it [00:15, 589976.23it/s]


Some of the sentences after preprocessing

In [8]:
with open('data/preprocessed.txt', 'r') as f:
    for i in range(5):
        print(next(f))

multifaceted covid19 russian experience according to current live statistics at the time of editing this letter russia has been the third country in the world to be affected by covid19 with both new cases and death rates rising

it remains in a position of advantage due to the later onset of the viral spread within the country since the worldwide disease outbreak

the first step in fighting the epidemic was nationwide lock down on march th

most of the multidisciplinary hospitals have been repurposed as dedicated covid19 centres so the surgeons started working as infectious disease specialists

such a reallocation of health care capacity results in the effective management of this epidemiological problem



## Train-Test Split

**The corpus containes more that 83L sentences due to computaional limitations I only use portion of data to build the language model**

In [5]:
def split_train_test(file_path, corpus_prop = 0.5, train_prop = 0.9):
    """
    args : file_path : path of file for preprocessed corpus
    returns : splits raw text to train and test set
    """
    
    """
    1 - count the number of lines
    2 - |train_set| = train_prop * number_of_lines
    3 - |test_set| = (1 - train_prop) * number_lines
    """
    
    train_file = 'data/train_preprocessed.txt'
    test_file = 'data/test_preprocessed.txt'
    print('getting line count')
    with open(file_path, 'r', encoding = 'utf-8') as f:
        for i, l in tqdm(enumerate(f)):
            pass
    num_lines =  i + 1
    print('Number of lines = {}'.format(num_lines))
    print('building train, test set using only {} % of the entire corpus'.format(corpus_prop*100))
    f1 = open(train_file, 'w', encoding = 'utf-8')
    f2 = open(test_file, 'w', encoding = 'utf-8')
    k = 0
    with open(file_path, 'r', encoding = 'utf-8') as f:
        for i, l in tqdm(enumerate(f)):
            if i < (num_lines*corpus_prop):
                k += 1
                if k < (num_lines*corpus_prop*train_prop):
                    f1.write(l)
                else:
                    f2.write(l)
    with open(train_file, 'r', encoding = 'utf-8') as f:
        for i, l in tqdm(enumerate(f)):
            pass
    print("Number of sentences in train set = {}".format(i+1))
    with open(test_file, 'r', encoding = 'utf-8') as f:
        for i, l in tqdm(enumerate(f)):
            pass
    print("Number of sentences in test set = {}".format(i+1))

In [6]:
file_path = 'data/preprocessed.txt'
split_train_test(file_path, train_prop=0.8)

102639it [00:00, 1019022.18it/s]

getting line count


8374537it [00:07, 1076872.17it/s]
49102it [00:00, 487461.81it/s]

Number of lines = 8374537
building train, test set using only 50.0 % of the entire corpus


8374537it [00:12, 691273.44it/s] 
3349782it [00:02, 1210793.59it/s]
238525it [00:00, 1192853.16it/s]

Number of sentences in train set = 3349782


837455it [00:00, 1203000.34it/s]

Number of sentences in test set = 837455


## Trigram Model

In [4]:
def build_trigram_model(file_path):
    trigram_model = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
    with open(file_path, 'r', encoding = 'utf-8') as f:
        lines = f.readlines()
        print('building trigram frequency dictionary')
        for l in tqdm(lines):
            for w1, w2, w3 in ngrams(l.split(' '),3, pad_right=True, pad_left=True):
                trigram_model[(w1, w2)][w3] += 1
        print('Normalizing frequncies to create probability distribution')
        for w1_w2 in tqdm(trigram_model):
            total_count = float(sum(trigram_model[w1_w2].values()))
            for w3 in trigram_model[w1_w2]:
                trigram_model[w1_w2][w3] /= total_count
    return trigram_model

In [5]:
train_file = 'data/train_preprocessed.txt'
trigram_model = build_trigram_model(train_file)

  0%|                                                                        | 1078/3349814 [00:00<05:12, 10702.34it/s]

building trigram frequency dictionary


  0%|                                                                         | 591/10460569 [00:00<52:54, 3294.59it/s]

Normalizing frequncies to create probability distribution


100%|██████████████████████████████████████████████████████████████████| 10460569/10460569 [00:16<00:00, 649805.95it/s]


Saving Model

In [ ]:
with open('data/trigram.csv', 'w', encoding ='utf-8') as f:
    writer = csv.writer(f)    
    writer.writerow(['w1_w2', 'w3_prob'])
    for k,v in tqdm(trigram_model.items()):
        l = [k]
        l.append(list(v.items()))
        writer.writerow(l)

 96%|███████████████████████████████████████████████████████████████▌  | 16568902/17220321 [03:10<00:03, 202692.78it/s]

## Finding most frequent starting word and generating a ramdom sentence from it

In [6]:
most_frequent_word =  max(trigram_model[None, None].items(), key=operator.itemgetter(1))[0]

In [7]:
most_frequent_word

'the'

In [11]:
text = [None, most_frequent_word]
sentence_finished = False
while not sentence_finished:
    # select a random probability threshold  
    r = random.random()
    accumulator = .0

    for word in trigram_model[tuple(text[-2:])].keys():
        accumulator += trigram_model[tuple(text[-2:])][word]
        # select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            break

    if text[-2:] == [None, None]:
        sentence_finished = True

print('Randomly generated sentence from the model')
print (' '.join([t for t in text if t]))

Randomly generated sentence from the model
the time of the virus multiply quickly in an n antigencapture assays detected a significantly higher level of depression



## Fourgram Model

In [7]:
def build_fourgram_model(file_path):
    fourgram_model = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
    with open(file_path, 'r', encoding = 'utf-8') as f:
        lines = f.readlines()
        print('building fourgram frequency dictionary')
        for i, l in tqdm(enumerate(lines)):
            for w1, w2, w3, w4 in ngrams(l.split(' '), 4, pad_right=True, pad_left=True):
                fourgram_model[(w1, w2, w3)][w4] += 1
            #if i == 100:
            #    break
        # print(list(fourgram_model.keys())[0])
        print('Normalizing frequncies to create probability distribution')
        for w1_w2_w3 in fourgram_model:
            total_count = float(sum(fourgram_model[w1_w2_w3].values()))
            for w4 in fourgram_model[w1_w2_w3]:
                fourgram_model[w1_w2_w3][w4] /= total_count
    return fourgram_model

In [8]:
train_file = 'data/train_preprocessed.txt'
fourgram_model = build_fourgram_model(train_file)

141it [00:00, 1017.10it/s]

building fourgram frequency dictionary


3349814it [20:26, 2730.39it/s] 


Normalizing frequncies to create probability distribution


## Finding most frequent starting word and generating a ramdom sentence from it

In [9]:
most_frequent_word_4gram =  max(fourgram_model[None, None, None].items(), key=operator.itemgetter(1))[0]

In [10]:
most_frequent_word_4gram

'the'

In [15]:
text = [None, None, most_frequent_word_4gram]
sentence_finished = False
while not sentence_finished:
    # select a random probability threshold  
    r = random.random()
    accumulator = .0

    for word in fourgram_model[tuple(text[-3:])].keys():
        accumulator += fourgram_model[tuple(text[-3:])][word]
        # select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            break

    if text[-3:] == [None, None, None]:
        sentence_finished = True

print('Randomly generated sentence from the 4gram model')
print (' '.join([t for t in text if t]))

Randomly generated sentence from the 4gram model
the latter recommendation is based on a cohort of hospitalized hypertensive covid19 patients controlling for rhv



## Calculating Perplexities to evaluate models

For evaluating the our models we'll use **perplexity metric**, given a sentence containing (w1, w2, w3, w4, ...), perplexity can be calculated as - 

<img src="figures/perplexity.png">, 

**There for maximizing probability is the same as minimizing perplexity**

In [9]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [3]:
tokenized_text = []
with open('data/train_preprocessed.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    tokenized_text = [list(line[:-1].split(' ')) for line in lines]

In [4]:
n = 3
train_data_trigrams, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
trigram_model = MLE(n)
trigram_model.fit(train_data_trigrams, padded_vocab)

In [4]:
tokenized_text_test = []
with open('data/test_preprocessed.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    tokenized_text_test = [list(line[:-1].split(' ')) for i, line in enumerate(lines) if (i < 50000 )]

In [6]:
len(tokenized_text_test)

50000

In [7]:
test_data_trigrams, _ = padded_everygram_pipeline(n, tokenized_text_test)

In [8]:
perplexity = 0
n_valid = 0
for i, test in tqdm(enumerate(test_data_trigrams)):
    try:
        p = trigram_model.perplexity(test)
        if p > 0 and (not math.isinf(p)):
            perplexity += p
            n_valid += 1
    except ZeroDevideError:
        continue

50000it [00:24, 2049.44it/s]


Average perplexity trigrams over 50000 valid test sentences

In [9]:
perplexity/n_valid

103.18152347921301

In [ ]:
n1 = 4
train_data_fourgrams, padded_vocab = padded_everygram_pipeline(n1, tokenized_text)
fourgram_model = MLE(n1)
fourgram_model.fit(train_data_fourgrams, padded_vocab)

In [ ]:
test_data_fourgrams, _ = padded_everygram_pipeline(n1, tokenized_text_test)

In [ ]:
perplexity = 0
n_valid = 0
for i, test in tqdm(enumerate(test_data_fourggrams)):
    try:
        p = fourgram_model.perplexity(test)
        if p > 0 and (not math.isinf(p)):
            perplexity += p
            n_valid += 1
    except ZeroDevideError:
        continue

Average perplexity for fourgram model for 50000 valid test sentences

In [1]:
perplexity/n_valid

83.2184359435987

We can see that perplexity of fourgram model is lower than that of trigram model, hence fourgram model performs better on the test set

## 6 -
An effitient way to handle large set of parameters is to use better datastructure with lower space complexity such as hash sets, etc.

## 7 -

While counting the frequencies of the last word in ngram model we can parallel proces that set, using hadoop clusters mainly by employing a technique known as map-reduce which is highly efficient in parallal processing and ditributing tasks over several systems